In [1]:
"""Installing all the relevant packages for testing."""
!pip install pycryptodome cryptography

Defaulting to user installation because normal site-packages is not writeable


In [2]:
"""Importing all necessary modules from pycryptodome and cryptography packages for the algorithm."""
from cryptography.hazmat.primitives import asymmetric, serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, DES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
from Crypto.Util.number import getPrime
from Crypto.Util.Padding import pad, unpad
import socket
import base64
import time
import hashlib

"""Creating a suitable class in order to encrypt and decrypt messages using the algorithms."""
class RSACrypto:
    def __init__(self, key_size=2048, public_exponent=65537, prime_size=1024, hash_function='SHA-256'):
        """Initializes all the evaluation parameters for the algorithm."""
        self.key_size = key_size
        self.key_size = key_size
        self.public_exponent = public_exponent
        self.hash_function = hash_function
        self.prime_size = prime_size
        self.RSA_generation_time = None
        self.DES_generation_time = None
        self.generate_keys()
        
    def generate_keys(self):
        """Generates RSA key pair with defined parameters."""
        start_time = time.time()
        key = RSA.generate(self.key_size, e=self.public_exponent)
        self.private_key = key.export_key()
        self.public_key = key.publickey().export_key()
        self.RSA_generation_time = time.time() - start_time
        
    def generate_AES_key(self):
        """Generates a single key for AES (Advanced Encryption Standard)."""
        start_time = time.time()
        key = get_random_bytes(16)
        end_time = time.time()
        self.AES_generation_time = end_time - start_time
        return key, self.AES_generation_time

    def generate_DES_key(self):
        """Generates a single key for DES (Data Encryption Standard)."""
        start_time = time.time()
        key = get_random_bytes(8)
        end_time = time.time()
        self.DES_generation_time = end_time - start_time
        return key, self.DES_generation_time
    
    def public_encrypt(self, message):
        """Encrypts a message using the public key."""
        if self.public_key is None:
            raise ValueError("Public key not generated or loaded.")
        key = RSA.import_key(self.public_key)
        cipher = PKCS1_OAEP.new(key)
        encrypted_message = cipher.encrypt(message.encode())
        return encrypted_message
    
    def private_decrypt(self, encoded_message):
        """Decrypts a message using the private key."""
        if self.private_key is None:
            raise ValueError("Private key not available.")
        
        key = RSA.import_key(self.private_key)
        cipher = PKCS1_OAEP.new(key)
        decrypted_message = cipher.decrypt(encoded_message)
        return decrypted_message.decode()

    def get_security_parameters(self):
        """Returns security-related parameters."""
        return {
            'Key Size': self.key_size,
            'Public Exponent': self.public_exponent,
            'Prime Size': self.prime_size,
            'Hash Function': self.hash_function,
            'RSA Key Generation Time': self.RSA_generation_time,
            'DES Key Generation Time': self.DES_generation_time
        }

In [3]:
"""Printing names of available hash functions."""
print(hashlib.algorithms_available)

{'sha512_256', 'mdc2', 'shake_256', 'sha1', 'sha512', 'shake_128', 'sha3_256', 'md5-sha1', 'blake2s', 'blake2b', 'ripemd160', 'sm3', 'whirlpool', 'md4', 'sha512_224', 'sha256', 'sha384', 'sha3_512', 'sha3_224', 'sha3_384', 'sha224', 'md5'}


In [4]:
"""Using if statement to develop user input prototype for testing the algorithms RSA, AES and DES."""
if __name__ == "__main__":
    rsa = RSACrypto(key_size=2048, hash_function='sha256')
    user_input = input("Please enter a message:")
    encrypted_message = rsa.public_encrypt(user_input)
    decrypted_message = rsa.private_decrypt(encrypted_message)
    print(encrypted_message)
    print(decrypted_message)
    print(rsa.get_security_parameters())
    key_1 = get_random_bytes(16)
    AES_instance = AES.new(key_1, AES.MODE_ECB)
    encrypted_AES = AES_instance.encrypt(pad(user_input.encode('utf-16'), AES.block_size))
    decrypted_AES = unpad(AES_instance.decrypt(encrypted_AES), AES.block_size)
    print(encrypted_AES)
    print(decrypted_AES.decode('utf-16'))
    print(rsa.generate_AES_key())
    difference_1 = rsa.RSA_generation_time - rsa.AES_generation_time
    print("Time Difference =",difference_1)
    print(f"RSA is a more secure algorithm than AES since the difference in generation time {difference_1}>0 implying that AES is faster and therefore less secure and AES will only accept two key parameters out of RSA (key size and generation time) since RSA generates prime numbers and incurs hash functions but AES does not do either of these.")
    key_2 = get_random_bytes(8)
    DES_instance = DES.new(key_2, DES.MODE_ECB)
    encrypted_DES = DES_instance.encrypt(pad(user_input.encode('utf-8'), DES.block_size))
    decrypted_DES = unpad(DES_instance.decrypt(encrypted_DES), DES.block_size)
    print(encrypted_DES)
    print(decrypted_DES.decode('utf-8'))
    print(rsa.generate_DES_key())
    difference_2 = rsa.RSA_generation_time - rsa.DES_generation_time
    print("Time Difference =",difference_2)
    print(f"RSA is a more secure algorithm than DES since the difference in generation time {difference_2}>0 implying that DES is faster and therefore less secure and DES will only accept two key parameters out of RSA (key size and generation time) since RSA generates prime numbers and incurs hash functions but DES does not do either of these.")

Please enter a message: Hello, World!


b"Ti\xbb.\x85\xaa\xee\x86\x10\xe8-v,F\x93w\x81mp.}F3\x86\x99\xe2H\xbe\xdfW5\xf7\x83\xed|0\xaa^\xdc)c\xe5W\x8f\x9cR.w\xc3\x87Y\xcdqt\xe1\xd00\x10i\xc5\x83\xd1\xbe\xf7\xa3\xc7\x11\x9d\xfe\x94+\xbd\xa3\xf0\xabh\x93\xf5,\x02\x86\xf7\xc4\xf3:\xd6J?\x1d\x8c\xe9\t\xf8#E\xfeL\xea\xb0R\xa6v\x83\n\x93\xb5wS\xd1\x93]\\\x18\xb55/k\x0eT\xb4\xbc>\x15\x87hiH\xb2r\xeaz\xf9\xde\x0f\xfaU\x88!\t\xac,:\xa5\x90NVG\xe8\t\xf1.\xd0\xe4\xe6\x12\xf6\xf1\xb4\xcc]\x07\xe1WI\x05\xbf\x80i+\xef\x91\xbb\x97\xc3OF\xd7^0\xdf\x12\xf1\xd1*\x03tb,\x19\xc5\xa9u\xaeaT{\tm\x13'\x95\xc3\xf1l\x0c\xb5\x85F5\x0bkD\x9c\xf5\xbf\xc7\xda\x8d\xa5\x8d\xf1}\xb6ipk\xf0\xaea`\xf1\x11\xc6m\x87\xe2\xe1\x83\xfe\xfd\x10\xe5\xf8O\xa8\xe8\xd1h\xea\xa22g\xc5\xfd\x91@"
Hello, World!
{'Key Size': 2048, 'Public Exponent': 65537, 'Prime Size': 1024, 'Hash Function': 'sha256', 'RSA Key Generation Time': 0.8213131427764893, 'DES Key Generation Time': None}
b'#\x9e\xfc\x01\x82d\xbc\xf3\x81\xf4X\xeb\xf3g\xeeH=\x9e\xd9|\x96cL8\x99\xf9\x13\x1bA\x12\x06\x

In [11]:
"""Using if statement to develop user input prototype for testing RSA and DES."""
if __name__ == "__main__":
    rsa = RSACrypto(key_size=2048, hash_function='sha256')
    user_input = input("Please enter a message:")
    encrypted_message = rsa.public_encrypt(user_input)
    decrypted_message = rsa.private_decrypt(encrypted_message)
    print(encrypted_message)
    print(decrypted_message)
    print(rsa.get_security_parameters())
    key_1 = get_random_bytes(16)
    AES_instance = AES.new(key_1, AES.MODE_ECB)
    encrypted_AES = AES_instance.encrypt(pad(user_input.encode('utf-16'), AES.block_size))
    decrypted_AES = unpad(AES_instance.decrypt(encrypted_AES), AES.block_size)
    print(encrypted_AES)
    print(decrypted_AES.decode('utf-16'))
    print(rsa.generate_AES_key())
    difference_1 = rsa.RSA_generation_time - rsa.AES_generation_time
    print("Time Difference =",difference_1)
    print(f"RSA is a more secure algorithm than AES since the difference in generation time {difference_1}>0 implying that AES is faster and therefore less secure and AES will only accept two key parameters out of RSA (key size and generation time) since RSA generates prime numbers and incurs hash functions but AES does not do either of these.")
    key_2 = get_random_bytes(8)
    DES_instance = DES.new(key_2, DES.MODE_ECB)
    encrypted_DES = DES_instance.encrypt(pad(user_input.encode('utf-8'), DES.block_size))
    decrypted_DES = unpad(DES_instance.decrypt(encrypted_DES), DES.block_size)
    print(encrypted_DES)
    print(decrypted_DES.decode('utf-8'))
    print(rsa.generate_DES_key())
    difference_2 = rsa.RSA_generation_time - rsa.DES_generation_time
    print("Time Difference =",difference_2)
    print(f"RSA is a more secure algorithm than DES since the difference in generation time {difference_2}>0 implying that DES is faster and therefore less secure and DES will only accept two key parameters out of RSA (key size and generation time) since RSA generates prime numbers and incurs hash functions but DES does not do either of these.")

Please enter a message: Hi, my name is Max Hennebry.


b'9b\xab\t\x81D\xbf\xef1\xd0]F\xad\x06\xe7\xcc\x1c\xb6\xa3j\x13D;PP\xac\xcb\x89\xbej\n\xe4\xfa\x9cO\x15\x04\xb7\x8f\x8f\xede\xc8\xb6\xd9\xfc\x9a^\x9e2\x98rA\xf0\xb2\x1c\x16[\xd4gj\x19z\x00\xf6\xbb\xe1\\\x85+\xe07z\xca\xb1r\xaa\xd9CnP\x0e\xe6\x9e\xb2x\x026\x13\xc2\x1b\xb1\xdf>\x7f\xf9S\x06\x1a\xe2d\x1b\xf3nt\xc3e\xf6R\xaa\xfd%\xff\xe81\x83OrF|\xe6@\xd4\xc8\xf0\xf1!*\xf6e\xd5\x07)q\x9b\x893:\x90"J)\x13\xbb))\x1d`\xac!\xd1\tL..w\x14\x1b\x03n7\xd3\t\\\x93F\xc0\x0f\x88\xe5\xb3\x1e\xf0:.\xc9Q<\xc9\x95\xa5\x9f\x84\xb8\x08UA\xda4\xae;4M\x97S\xb5\x17,|\xcfLb\xd7\x9f\x01\xdc"\xc3*"\x07b\x0c\xb6\x1eJPB\xd5\xc2P8c|#u5\xfa\'Od0\xb3\xf4\xde@\xfc\xe6c`\x16s\xf7%\xf4}\x14\xa2\x90\x96S\xd7\xc5\xe6\x83\xc3'
Hi, my name is Max Hennebry.
{'Key Size': 2048, 'Public Exponent': 65537, 'Prime Size': 1024, 'Hash Function': 'sha256', 'RSA Key Generation Time': 3.6124701499938965, 'DES Key Generation Time': None}
b'2\xa9G\x14)\x97=\x82?\x0b\x13C\x9cY-\xd1+\xbb\xf4\xe1\xad\xb3\xc0\x9e\xcf\x1f+\xc1\'\x91\xa5\t;\xe

In [13]:
"""Using if statement to develop user input prototype for testing RSA and DES."""
if __name__ == "__main__":
    rsa = RSACrypto(key_size=2048, hash_function='sha256')
    user_input = input("Please enter a message:")
    encrypted_message = rsa.public_encrypt(user_input)
    decrypted_message = rsa.private_decrypt(encrypted_message)
    print(encrypted_message)
    print(decrypted_message)
    print(rsa.get_security_parameters())
    key_1 = get_random_bytes(16)
    AES_instance = AES.new(key_1, AES.MODE_ECB)
    encrypted_AES = AES_instance.encrypt(pad(user_input.encode('utf-16'), AES.block_size))
    decrypted_AES = unpad(AES_instance.decrypt(encrypted_AES), AES.block_size)
    print(encrypted_AES)
    print(decrypted_AES.decode('utf-16'))
    print(rsa.generate_AES_key())
    difference_1 = rsa.RSA_generation_time - rsa.AES_generation_time
    print("Time Difference =",difference_1)
    print(f"RSA is a more secure algorithm than AES since the difference in generation time {difference_1}>0 implying that AES is faster and therefore less secure and AES will only accept two key parameters out of RSA (key size and generation time) since RSA generates prime numbers and incurs hash functions but AES does not do either of these.")
    key_2 = get_random_bytes(8)
    DES_instance = DES.new(key_2, DES.MODE_ECB)
    encrypted_DES = DES_instance.encrypt(pad(user_input.encode('utf-8'), DES.block_size))
    decrypted_DES = unpad(DES_instance.decrypt(encrypted_DES), DES.block_size)
    print(encrypted_DES)
    print(decrypted_DES.decode('utf-8'))
    print(rsa.generate_DES_key())
    difference_2 = rsa.RSA_generation_time - rsa.DES_generation_time
    print("Time Difference =",difference_2)
    print(f"RSA is a more secure algorithm than DES since the difference in generation time {difference_2}>0 implying that DES is faster and therefore less secure and DES will only accept two key parameters out of RSA (key size and generation time) since RSA generates prime numbers and incurs hash functions but DES does not do either of these.")

Please enter a message: How I wish I could calculate pi.


b'NB\xa4i\xc76\'\xc9\xe3$~\x9d\xee\x9b\x85F\xe4\xfe\x1c\xd3\xcf\xcb_\x14\x08\xd0\xddN\x9fG\x9aB%J<\xda\x1df\x93]W\x1e\xa8W\xa5\x15Xdt4#\x8c\x81\\!0\xfd\xff^\xec\xa7\xca\xe7)\x8c|\xb5\xad\xa9\xb9\xac1\x99\x03\x90_\xe8\x1a\xb9\x85Z\xed\xfbFn_\xf0R1\xef(}w[\xe5\x9fTF\x90\xd9\xeaRH\xb5\xa8\xfbKQ\x16\x9c\\\x1a\x00\xb0!\xf5Q\xe7\xeb\x12\xee7\xd9\xa5\xab\xa7\xa8$\xc9\x15\x94T\xc7de@\xe5^\xe6\xca\x85\x1b/\xe3\x0eXlU\xc7\xe8\xc4\xcf\x05\xaeA\x1a\x05e\xaa\xb5\xdfF&\x01EkH\x86\xdd\x7f\x8f?\xdb\x0c\xa9\xaf\r\xdej\xa78\x16\xb3\x99\x0cZ\xe6"\x8a\xfe*\x1c\xa1\xa0\x8e\xb7sj\xc2\xc1.\xbe\xe3\x98<o\xf7GE\x14\xd3\xe3\xcb\x82\xef#QA\x88&\xdc\xb8x\xaf\x94\x9b\xe5;\xdc\x9ft\x86\xae8\xd7\xd6\xeeTR\xfe\xc4LD\x19\x87bn\xa5\x06T\xb2\xed\x18\x0f\xb5\x83'
How I wish I could calculate pi.
{'Key Size': 2048, 'Public Exponent': 65537, 'Prime Size': 1024, 'Hash Function': 'sha256', 'RSA Key Generation Time': 0.7214579582214355, 'DES Key Generation Time': None}
b'z\xa5b\x88\xf4\xe2.\xa3\x0b\x15\x1e\x91\xb4\xc6s\xcb\xa

In [15]:
"""Using if statement to develop user input prototype for testing RSA and DES."""
if __name__ == "__main__":
    rsa = RSACrypto(key_size=2048, hash_function='sha256')
    user_input = input("Please enter a message:")
    encrypted_message = rsa.public_encrypt(user_input)
    decrypted_message = rsa.private_decrypt(encrypted_message)
    print(encrypted_message)
    print(decrypted_message)
    print(rsa.get_security_parameters())
    key_1 = get_random_bytes(16)
    AES_instance = AES.new(key_1, AES.MODE_ECB)
    encrypted_AES = AES_instance.encrypt(pad(user_input.encode('utf-16'), AES.block_size))
    decrypted_AES = unpad(AES_instance.decrypt(encrypted_AES), AES.block_size)
    print(encrypted_AES)
    print(decrypted_AES.decode('utf-16'))
    print(rsa.generate_AES_key())
    difference_1 = rsa.RSA_generation_time - rsa.AES_generation_time
    print("Time Difference =",difference_1)
    print(f"RSA is a more secure algorithm than AES since the difference in generation time {difference_1}>0 implying that AES is faster and therefore less secure and AES will only accept two key parameters out of RSA (key size and generation time) since RSA generates prime numbers and incurs hash functions but AES does not do either of these.")
    key_2 = get_random_bytes(8)
    DES_instance = DES.new(key_2, DES.MODE_ECB)
    encrypted_DES = DES_instance.encrypt(pad(user_input.encode('utf-8'), DES.block_size))
    decrypted_DES = unpad(DES_instance.decrypt(encrypted_DES), DES.block_size)
    print(encrypted_DES)
    print(decrypted_DES.decode('utf-8'))
    print(rsa.generate_DES_key())
    difference_2 = rsa.RSA_generation_time - rsa.DES_generation_time
    print("Time Difference =",difference_2)
    print(f"RSA is a more secure algorithm than DES since the difference in generation time {difference_2}>0 implying that DES is faster and therefore less secure and DES will only accept two key parameters out of RSA (key size and generation time) since RSA generates prime numbers and incurs hash functions but DES does not do either of these.")

Please enter a message: The quick brown fox jumps over the lazy dog.


b"x!f{\x93C\x9f\xf9\x11\x94\xf9\xbe\xa9EV\xa0\xf6N\xaa\x94\x92\x00\xa7'\xcd\xd1\xcd\xea_F\xd7\xcdg\xfek9\x86\x8a[[\xfe\xe7Y\xa0\xd0s\xbd&8\x15\xe8\xdfr#[O\xd2\xd7\xdaBim\x80V\x8f\x01\xec\x95v\xe2X\x0b\xc4KU\r\x1f\xed\xc4t\xb6\xc4*{\xac\xf9\x96C\xaco5O\xbaF\x00\xcd\xd9\xf6\xb6\xdb\xf3\xb7\xbc\x8e\xdb\x05\x19\x18`\xad\xee+]\x9c\x9d\xc1\x0f\t\x9a\x8dE\x15\xb1\xc1\xca\xec\xd8\xa9\x80\xa4u\x98n\xc0\x95\xe1\xe54C\xb9n\xa7Hk\xe4\x1a\x86\xf2e5r\xbe\r#\xcad\x19\xa0\xea\x1f~\x81\xeeu-\xfc\xdd\x17C2\x92z\x16\th\xe1\xe9B\xa5w\x01\x1agV\x0e\x16\xc9\xd5\xefs\x03>\x07\x86\xa1TS\xa7e\xa1aUT\x88 \x05(\xbf\xad\xc7\xdc\x14\xc4\x8a\x92\x95\x11\xf5(G\x10ED\x10G:\x12Q]7k,\xaf\x90\xd4\xb7\xeaf\x8b)@z\xfbg\xe6{p\xc7c\\x\n\xd9e\xb5\x95"
The quick brown fox jumps over the lazy dog.
{'Key Size': 2048, 'Public Exponent': 65537, 'Prime Size': 1024, 'Hash Function': 'sha256', 'RSA Key Generation Time': 3.232954263687134, 'DES Key Generation Time': None}
b'\xc3\r\xf5l(\x8b_\x9e\xe7Q\x82\xdf\xa6\x93\xeb\x82i4>\x14Z\x

In [17]:
"""Using if statement to develop user input prototype for testing RSA and DES."""
if __name__ == "__main__":
    rsa = RSACrypto(key_size=2048, hash_function='sha256')
    user_input = input("Please enter a message:")
    encrypted_message = rsa.public_encrypt(user_input)
    decrypted_message = rsa.private_decrypt(encrypted_message)
    print(encrypted_message)
    print(decrypted_message)
    print(rsa.get_security_parameters())
    key_1 = get_random_bytes(16)
    AES_instance = AES.new(key_1, AES.MODE_ECB)
    encrypted_AES = AES_instance.encrypt(pad(user_input.encode('utf-16'), AES.block_size))
    decrypted_AES = unpad(AES_instance.decrypt(encrypted_AES), AES.block_size)
    print(encrypted_AES)
    print(decrypted_AES.decode('utf-16'))
    print(rsa.generate_AES_key())
    difference_1 = rsa.RSA_generation_time - rsa.AES_generation_time
    print("Time Difference =",difference_1)
    print(f"RSA is a more secure algorithm than AES since the difference in generation time {difference_1}>0 implying that AES is faster and therefore less secure and AES will only accept two key parameters out of RSA (key size and generation time) since RSA generates prime numbers and incurs hash functions but AES does not do either of these.")
    key_2 = get_random_bytes(8)
    DES_instance = DES.new(key_2, DES.MODE_ECB)
    encrypted_DES = DES_instance.encrypt(pad(user_input.encode('utf-8'), DES.block_size))
    decrypted_DES = unpad(DES_instance.decrypt(encrypted_DES), DES.block_size)
    print(encrypted_DES)
    print(decrypted_DES.decode('utf-8'))
    print(rsa.generate_DES_key())
    difference_2 = rsa.RSA_generation_time - rsa.DES_generation_time
    print("Time Difference =",difference_2)
    print(f"RSA is a more secure algorithm than DES since the difference in generation time {difference_2}>0 implying that DES is faster and therefore less secure and DES will only accept two key parameters out of RSA (key size and generation time) since RSA generates prime numbers and incurs hash functions but DES does not do either of these.")

Please enter a message: Hi, my name is Sebastian Xavier Maximilien Hennebry and I am a Masters' student in Cybersecurity at the University of Adelaide.


b'2\x8aU\x94\x0b(\n5J\x99`\x1e\x03r\x1c\xa87\x11\x8c]\x01\x1f\xa4\\\xd9f\xdc_\x19\x02N\xb8H\x8c\xfd\xb1\\\xa9Z\xc2\x18\x8a\xda\xe3\xf3y"\xb7G\xab\xfbZ:\xfdN\xa0K>\x14N\x9b\x13\xe1\xdd\xdf\x19\xc4\xcc\x0b\xc8\xbb\xb3\xd95\x8a\x94\x8eP(Y\x1dI\x85\xfeF\xe2(\x1a\x8a\x992\xda\x12\xe5\xc2$] \x92\x0ff\xf2\x0b]\xc1\xb1hc\xb1z\x7f|\xb5\xd1O\x0c\xa7\xb0u%4Q~R3\xd7\x11:\x1a\xe9\xe4}\xf48\xb7\x8b\xa0\x98\xd8%\xe3\xa2\x16\xe3\xe0_\r\x0e=tz\xf80^]\xc1\x00\xd1\x92b\x8a\x0c\x98\xfa\xb4\x81=\x01*\xde\xa5\xc9\xf4)\xe4W\x91\xe3\xb1(\x80\xc7\x15\xa9G\x80g6)\xe8r\x82\xe6\xdb*\xf3\xde\xeb\xb2)\xd6\x8a\x02\x94\x17\x99\x97=\xc1P>\xce\xea:%\x83\x85\xf8\xd3y\xf8\x9b\x95F\xe7\x94\xef\xf1\xa3\x98t\xc3\xdd\t_\x7f\x0c\xd5t\xc0\xf7\xcb>\x8d+jLY\x19Uf\x16\xb3\xf3\x7fD'
Hi, my name is Sebastian Xavier Maximilien Hennebry and I am a Masters' student in Cybersecurity at the University of Adelaide.
{'Key Size': 2048, 'Public Exponent': 65537, 'Prime Size': 1024, 'Hash Function': 'sha256', 'RSA Key Generation Time': 5.185